In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install libais
!{sys.executable} -m pip install bokeh


In [2]:
import re
import pandas as pd
import numpy as np
import time

import ais

from dask.distributed import Client
from dask import dataframe as ddf
from dask.multiprocessing import get
from multiprocessing import cpu_count

nCores = cpu_count()

start = time.time()
start

1585223707.080263

In [3]:
def meta_parse(row):
    '''
    Parse meta strings. Either single or multiline headers:
    s:66,c:1555624754*3E
    or 
    g:1-2-0300,s:66,c:1555624791*46
    or 
    g:2-2-0300*5E
    and s:xx multiline messages?
    '''
    meta, meta_checksum = row['meta'].split("*")
    time_found = re.search(r'[0-9]{10}',meta)
    
    if time_found:
        event_time = time_found.group(0)
    else:
        event_time = None
    
    row['meta_checksum'] = meta_checksum
    row['meta_source'] = meta
    row['event_time'] = event_time
    
    return row


In [4]:
def ais_decode_to_dict(row, drop_message_type):
    try:
        decode_dict = ais.decode(row['payload'],int(row['padding'])) 
    except Exception as err:
        "try padding to reach 70"
        try:
            decode_dict = ais.decode(row['payload'], 12)
#             print('It worked!')
        except Exception as err:
#             print('{1} Still bad: {0}'.format(err, row.name))
#             print(row['ais'])            
            decode_dict = {}
            
    if decode_dict.get('id') in drop_message_type:
#         print('Dropping unwanted decoded message')
        decode_dict = {}
    return decode_dict

In [5]:
# 170 Lines of AIS in the expected format:
# infile = 'full_test.log'

# 10% of the daily file. Around 70 megs/ 800K messages. 
#infile = 'decimate_test.log'

# Full day:
infile = 'TNPA_AIS.log.2019-04-19'

ais_sentence = ['packet',
  'frag_count',
  'frag_num',
  'seq_id',
  'radio_chan',
  'payload',
  'padding',
  'checksum']

first_cols = ['rx_time',
          'meta',
          'ais',
          'meta_checksum',
          'meta_source',
          'event_time',
          'decoded',
          'msg_num']

meta_cols = ['meta_checksum',
            'meta_source',
            'event_time']

drop_message_type = [7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27]


# Prepare Data

## Using Pandas

In [6]:
%%time

df = pd.read_csv(infile,
                 sep=r": \\|\\",
                 names=first_cols, 
                 header=None,
                 skiprows=1, 
                 skip_blank_lines=True,
                 engine="python")
 
 

CPU times: user 39.1 s, sys: 2.09 s, total: 41.2 s
Wall time: 41.2 s


In [7]:
%%time

df[ais_sentence] = df['ais'].str.split(r',|\*', expand=True) 
df = df.apply(lambda x : meta_parse(x), axis=1)

CPU times: user 19min 2s, sys: 18.8 s, total: 19min 20s
Wall time: 19min 47s


## Using Dask

In [8]:
# %%time
# client = Client()
# client

In [9]:
# %%time 
 
# dask_dataframe = ddf.from_pandas(df, npartitions=2000*nCores)

In [10]:
# %%time
# dask_dataframe = dask_dataframe.apply(lambda x : meta_parse(x), axis=1, meta = dask_dataframe)
# dask_dataframe.compute()

In [11]:
# %%time
# single_dataframe = dask_dataframe[dask_dataframe.frag_count == '1']
# single_dataframe.compute()

In [12]:
# %%time
# multi_ddf = dask_dataframe[dask_dataframe.frag_count == '2']
# multi_df = multi_ddf.compute()

# Seperate in single and multi-line messages

In [13]:
%%time

multi_df = df[df.frag_count == '2']
multi_df['prev_payload'] = multi_df['payload'].shift(1)
multi_df['prev_event_time'] = multi_df['event_time'].shift(1)
multi_df['payload'] = multi_df['prev_payload'] + multi_df['payload']

multi_df =  multi_df[multi_df['prev_event_time'].notnull() & multi_df['meta_source'].str.contains('g:2')]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

CPU times: user 5.71 s, sys: 1.14 s, total: 6.85 s
Wall time: 6.96 s


In [14]:
%%time

# pass DF to row 
yy = multi_df.apply(lambda x : ais_decode_to_dict(x, drop_message_type), axis=1)
yy_df = pd.DataFrame(yy.values.tolist())

CPU times: user 4.3 s, sys: 11.9 ms, total: 4.32 s
Wall time: 4.37 s


In [ ]:
%%time

xx =  df.apply(lambda x : ais_decode_to_dict(x,drop_message_type),axis=1) 
# client.shutdown()

In [16]:
# client.shutdown()

In [17]:
%%time
new_df = pd.DataFrame(xx.values.tolist())
new_df.columns

NameError: name 'xx' is not defined

In [18]:
# new_df.to_csv('decoded_single.csv')
# yy_df.to_csv('decoded_multi.csv')
# multi_df.to_csv('parsed_multi.csv')

In [19]:
end = time.time()
print("Duration to process: {0} mins".format(round((end-start)/60,2)))

Duration to process: 20.68 mins


## To Do:
- DB Structure Pos vs Voy?   
- Combine Single and Multi?     
- DB larger when ?


# Use Cases (from experience)

- History of vessel X?
- What vessels are in this area?
- get point matches (sar-to-ais and radar-to-ais)
- Transhipments (ship-to-ship or ship-to-bilge)
- Get names/imo/callsigns from AIS pos report
- heatmap agg

# Reasearch use cases
- various distributions (speeds, nav status, courses) for vessel x for last 7 days (timescale db contiuous agg)
- Heatmap agg points
- Event based history
- API interface (although that's a seperate thing)
    